In [ ]:
import numpy as np
# import matplotlib.pyplot as plt
from functools import partial

In [ ]:
ifC=[-.03+i*.01 for i in range(7)]
ifL=['NL','NM','NS','AZ','PS','PM','PL']
ifD={fv:ifC[i] for i,fv in enumerate(ifL)}

In [7]:
thC=[-0.4,-0.2,-0.1,0.0,0.1,0.2,0.4]
thL=['SB','SM','SS','N','BS','BM','BB']
thD={fv:thC[i] for i,fv in enumerate(thL)}

In [ ]:
# Fuzzy function with centre and width (delta) in the mid or end of a range of fuzzy functions.
def fuzzy(x,center=0.0,delta=1.0,shape='mid'):
    y =0 
    if shape=='mid':
        if x>=center-delta and x<=center+delta:
            y = 1 - abs(x-center)/delta
    elif shape=='top':
        if x>=center-delta and x<=center:
            y = (x-center+delta)/delta
        elif x<center-delta: y = 0
        else: y = 1
    elif shape=='bot':
        if x<=center+delta and x>=center:
            y = (center+delta-x)/delta
        elif x>center+delta: y=0
        else: y=1
    return y

In [ ]:
# List of fuzzy functions (partials) with specified centres and deltas.
def condfuncs(centers,deltas,bot=True,top=True):
    funclist = []
    for i in range(len(centers)):
        if i==0 and bot: shape='bot'
        elif i==len(centers)-1 and top: shape='top'
        else: shape='mid'
        funclist+=[partial(fuzzy,center=centers[i],delta=deltas[i],shape=shape)]
    return funclist

Fuzzy function $H(X)$ obtained by applying the rule "IF $F(x)$ THEN $G(X)$" where $F(x)$ is an input fuzzy function and $G(x)$ a output fuzzy function with centre $g$:

$H(X) = \frac{g F(X)}{F(X)}$

Similarly for conjunction of rules "IF $F_i(X)$ THEN $G_i(X)$":

$H(X) = \frac{\sum_i g_i F_i(X)}{\sum_i F_i(X)}$

In [ ]:
# Fuzzy function obtained by the rule applying thCL weights to the fuzzy functions in fL 
def rule(x,thCL=None,fL=None):
    num=0.0
    den=0.0
    for i in range(len(thCL)):
        num+=thCL[i]*fL[i](x)
        den+=fL[i](x)
    if den<=1e-7 : den=.0001
    return num/den

In [13]:
class Fuzzy():
    def __init__(self):
        self.ifC=[-.03+i*.01 for i in range(7)]
        self.ifL=['NL','NM','NS','AZ','PS','PM','PL']
        self.ifD={fv:ifC[i] for i,fv in enumerate(ifL)}
        self.thC=[-0.4,-0.2,-0.1,0.0,0.1,0.2,0.4]
        self.thL=['SB','SM','SS','N','BS','BM','BB']
        self.thD={fv:thC[i] for i,fv in enumerate(thL)}
# Fuzzy function obtained by the rule applying thCL weights to the fuzzy functions in fL 
    def fuzzyrule(self,x,ifL,thenL,bot=True,top=True,ifscale=1,thscale=1):
        if len(ifL)!=len(thenL): raise Exception("if and then lengths mismatch")
        deltaL=[.01*ifscale]*len(ifL)
        centerL=[self.ifD[c]*ifscale for c in ifL]
        fL=condfuncs(centerL,deltaL,bot,top)
        thCL=[self.thD[t]*thscale for t in thenL]
        num=0.0
        den=0.0
        for i in range(len(thCL)):
            num+=thCL[i]*fL[i](x)
            den+=fL[i](x)
        if den<=1e-7 : den=.0001
        return num/den
    def dblrule(self,x,y,ifL1,thL1,ifL2,thL2,ifscale=1,thscale=1):
        if len(ifL1)!=len(thL1) or len(ifL2)!=len(thL2): 
            raise Exception("if and then lengths mismatch")
        deltaL1=[.01*ifscale]*len(ifL1)
        deltaL2=[.01*ifscale]*len(ifL2)
        centerL1=[self.ifD[c]*ifscale for c in ifL1]
        centerL2=[self.ifD[c]*ifscale for c in ifL2]
        fL1=condfuncs(centers=centerL1,deltas=deltaL1)
        fL2=condfuncs(centers=centerL2,deltas=deltaL2)
        thCL1=[self.thD[t]*thscale for t in thL1]
        thCL2=[self.thD[t]*thscale for t in thL2]
        num1,num2=0.0,0.0
        den1,den2=0.0,0.0
        for i in range(len(thCL1)):
            num1+=thCL1[i]*fL1[i](x)
            den1+=fL1[i](x)
        for i in range(len(thCL2)):
            num2+=thCL2[i]*fL2[i](y)
            den2+=fL2[i](x)
        if den1<=1e-7 : den1=.0001
        if den2<=1e-7 : den2=.0001
        return (num1+num2)/(den1+den2)

In [1]:
# Fuzzy function obtained by the rule applying thCL1 weights to the fuzzy functions in fL1
# and thCL2 weights to the fuzzy functions in fL2.
def dblrule(x,y,thCL1=None,thCL2=None,fL1=None,fL2=None):
    num1,num2=0.0,0.0
    den1,den2=0.0,0.0
    for i in range(len(thCL1)):
        num1+=thCL1[i]*fL1[i](x)
        den1+=fL1[i](x)
    for i in range(len(thCL2)):
        num2+=thCL2[i]*fL2[i](y)
        den2+=fL2[i](x)
    if den1<=1e-7 : den1=.0001
    if den2<=1e-7 : den2=.0001
    return (num1+num2)/(den1+den2)